In [ ]:
import os
from pathlib import Path
from glob import glob
from pprint import pprint

import numpy as np
import torch

from scipy import ndimage
import cv2
from skimage import measure, segmentation, morphology
from microfilm.microplot import microshow
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

## User Interface

This notebook walks you through five steps to clean your dataset. Previews of the changes should be made on a "dev dataset" which can be a manual copy-paste of some images in a file structure similar to that of the actual dataset. Statistics are collected on these to help guide the desired parameters for later cleaning steps. Once you've set them based on the dev dataset, they will be used to process the full dataset in the background. Because of this, **please make sure your dev dataset is a good representative sample of the actual DATA**. These steps are listed below:
1. Dataset setup: set the datasets' paths, the folder depth at which to aggregate the images into a common tensor for use at training/inference time, and the filenames for each channel and what their respective positions should be in the processed data.
2. Cell segmentation: no parameters here at the moment, feel free to run on the full dataset once you've seen the preview on the dev dataset.
3. Mask post-processing: removing cells that were cut off by the image edge and merging cells that were incorrectly split based on dialating their nuclei.
4. Filtering cells based on size: delete small cells and crop the remaining images based on the size distribution of the cell bounding boxes.
5. Normalization: based on a variety of possible choices

## Dataset Setup

In [ ]:
DATA_DIR = Path("/data/ishang/CCNB1-dataset/") # NEEDS TO BE ABSOLUTE PATH
DEV_DATA_DIR = Path("/home/ishang/HPA-embedding/dev-dataset") # NEEDS TO BE ABSOLUTE PATH
# TODO this should work even if they are not pngs...
CHANNEL_NAMES = ["cyclinb1", "microtubule", "nuclei"] # these are the names of the channels in the order they appear in the image, the names should be the same as the file name (without the .png extension)
# the below are channels whose indices we need to know explicitly
DAPI = 2
TUBL = 1
ANLN = None

In [ ]:
DATA_STR, DATA_FOLDER_PATH  = str(DATA_DIR), "data-folder.txt"
DEV_DATA_STR, DEV_DATA_FOLDER_PATH  = str(DEV_DATA_DIR), "dev-data-folder.txt"

In [ ]:
# first number in the output is the total number of images found and the second is the list of folders underwhich all images will be aggregated in the dev dataset
# in this case we automatically aggregate everything at the first level of depth in the dataset, if you want to group by well, experiment, etc. please add the lists of absolute paths to the "*data-folder.txt" files accordingly
!find $DATA_STR -type d > $DATA_FOLDER_PATH
!sed -i '1d' $DATA_FOLDER_PATH
!cat $DATA_FOLDER_PATH | wc -l
!find $DEV_DATA_STR -type d > $DEV_DATA_FOLDER_PATH
!sed -i '1d' $DEV_DATA_FOLDER_PATH
!cat $DEV_DATA_FOLDER_PATH

In [ ]:
# below is an example of the level of the file hierarchy at which the data will be aggregated for the dev-dataset
def image_paths_from_folders(folders_file):
    image_paths = list(open(folders_file, "r"))
    image_paths = [Path(x.strip()) for x in image_paths]
    return image_paths
image_paths = image_paths_from_folders(DATA_FOLDER_PATH)
dev_image_paths = image_paths_from_folders(DEV_DATA_FOLDER_PATH)
pprint(dev_image_paths)

## Segmentation

### DO NOT RERUN THIS CELL BELOW!!! YOU WILL HAVE TO KILL THE PROCESS AND RESTART THE KERNEL

In [ ]:
gpu_num = 7
device = f"cuda:{gpu_num}" if torch.cuda.is_available() else "cpu"
print(f"Running on device: {device}")

In [ ]:
# Note that the source change warning below is expected
pwd = Path(os.getcwd())
NUC_MODEL = pwd / "HPA-Cell-Segmentation" / "nuclei-model.pth"
CELL_MODEL = pwd / "HPA-Cell-Segmentation" / "cell-model.pth"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
segmentator = cellsegmentator.CellSegmentator(
    str(NUC_MODEL), str(CELL_MODEL), device=device, padding=True, multi_channel_model=(ANLN is not None)
)

### DO NOT RERUN THIS ^^^^^ YOU WILL HAVE TO KILL THE PROCESS AND RESTART THE KERNEL

In [ ]:
def merge_missing(targets, elements):
    incomplete_elements = np.unique(elements[~np.isin(elements, targets)]) # these are cell masks
    incomplete_elements = incomplete_elements[incomplete_elements != 0]
    target_objects = np.stack([targets == nucleus for nucleus in np.unique(targets)])
    target_coms = np.array([ndimage.center_of_mass(nucleus_mask) for nucleus_mask in target_objects])
    for element in incomplete_elements:
        # get the closest nucleus
        element_com = ndimage.center_of_mass(elements == element)
        target_distances = np.linalg.norm(target_coms - element_com, axis=1)
        closest_target = np.unique(targets[target_distances == np.min(target_distances)])[0]
        # replace the missing nucleus with the closest nucleus
        elements[elements == element] = closest_target
    return elements

def get_masks(segmentator, image_paths, merge_missing=True):
    """
    Get the masks for the images in image_paths using HPA-Cell-Segmentation
    It places all the cells in the target directory from image_paths into a single
    """
    images_paths = []
    nuclei_mask_paths = []
    cell_mask_paths = []
    for image_path in image_paths:
        if (image_path / "images.npy").exists():
            images_paths.append(image_path / "images.npy")
        if ((image_path / "cell_masks.npy").exists() and
            (image_path / "nuclei_masks.npy").exists()):
            nuclei_mask_paths.append(image_path / "nuclei_masks.npy")
            cell_mask_paths.append(image_path / "cell_masks.npy")
            continue
        glob_channel_images = lambda image_path, c: list(glob(f"{str(image_path)}/**/*{CHANNEL_NAMES[c]}.png", recursive=True))
        dapi_paths = sorted(glob_channel_images(image_path, DAPI))
        tubl_paths = sorted(glob_channel_images(image_path, TUBL))
        anln_paths = sorted(glob_channel_images(image_path, ANLN)) if ANLN is not None else None
        
        for dapi, tubl in zip(dapi_paths, tubl_paths):
            assert str(dapi).split(CHANNEL_NAMES[DAPI])[0] == str(tubl).split(CHANNEL_NAMES[TUBL])[0], f"File mismatch for {dapi} and {tubl}"
        if ANLN is not None and anln_paths is not None:
            for dapi, anln in zip(dapi_paths, anln_paths):
                assert str(dapi).split(CHANNEL_NAMES[DAPI])[0] == str(anln).split(CHANNEL_NAMES[ANLN])[0], f"File mismatch for {dapi} and {anln}"

        load_image = lambda path_list: [cv2.imread(str(x), cv2.IMREAD_UNCHANGED) for x in path_list]
        dapi_images = load_image(dapi_paths)
        tubl_images = load_image(tubl_paths)
        anln_images = load_image(anln_paths) if anln_paths is not None else None

        ref_images = [tubl_images, anln_images, dapi_images]
        nuc_segmentation = segmentator.pred_nuclei(ref_images[2])
        cell_segmentation = segmentator.pred_cells(ref_images)

        # post-processing
        nuclei_masks, cell_masks = [], []
        for i in range(len(ref_images[2])): # 2 because DAPI will always be present and we set the order manually
            nuclei_mask = label_nuclei(nuc_segmentation[i])
            nuclei_mask, cell_mask = label_cell(
                nuc_segmentation[0], cell_segmentation[i]
            )
            nuclei_masks.append(nuclei_mask)
            cell_masks.append(cell_mask)
        nuclei_masks = np.stack(nuclei_masks, axis=0)
        cell_masks = np.stack(cell_masks, axis=0)

        # apply preprocessing mask if the user want to merge nuclei
        # in preprocess_masks get_single_cell_mask
        # TODO get COM for nuclei and cells, and for either without a counterpart add it to the id of the closest COM of the other type
        # TODO how are multi-nucleated cells handled?
        images = []
        for c, channel in enumerate(CHANNEL_NAMES):
            channel_paths = sorted(glob_channel_images(image_path, c))
            channel_images = load_image(channel_paths)
            channel_images = np.stack(channel_images, axis=0)
            images.append(channel_images)
        images = np.stack(images, axis=1)
        for i in [0, -2, -1]:
            assert images.shape[i] == nuclei_masks.shape[i] == cell_masks.shape[i], f"Shape mismatch for images and masks in {image_path}, at index {i}, images has shape {images.shape}, nuclei_masks has shape {nuclei_masks.shape}, cell_masks has shape {cell_masks.shape}"
        image_idx = 0
        for image, nuclei_mask, cell_mask in zip(images, nuclei_masks, cell_masks):
            if set(np.unique(nuclei_mask)) != set(np.unique(cell_mask)): 
                print(f"Mask mismatch for {image_path}, nuclei: {np.unique(nuclei_masks)}, cell: {np.unique(cell_masks)}")
                microshow(image[(DAPI, TUBL),], cmaps=["pure_blue", "pure_red"], label_text=f"Image: {Path(image_path).name}[{image_idx}] ")
                missing_nuclei = np.asarray(list(set(np.unique(cell_mask)) - set(np.unique(nuclei_mask))))
                if len(missing_nuclei) > 0:
                    microshow(image[TUBL] * np.isin(cell_mask, missing_nuclei), cmaps=["pure_red"], label_text=f"Cells missing nuclei: {missing_nuclei}")
                missing_cells = np.asarray(list(set(np.unique(nuclei_mask)) - set(np.unique(cell_mask))))
                if len(missing_cells) > 0:
                    microshow(image[DAPI] * np.isin(nuclei_mask, missing_cells), cmaps=["pure_blue"], label_text=f"Nuclei without cells: {missing_cells}")
                microshow(cell_mask, label_text=f"Cell mask: {np.unique(cell_mask)}")
                if len(missing_nuclei) > 0 and merge_missing:
                    cell_mask = merge_missing(nuclei_mask, cell_mask)
                    microshow(cell_mask, label_text=f"Cell mask after merging: {np.unique(cell_mask)}")
                microshow(nuclei_mask, label_text=f"Nuclei mask: {np.unique(nuclei_mask)}")
                if len(missing_cells) > 0 and merge_missing:
                    nuclei_mask = merge_missing(cell_mask, nuclei_mask)
                    microshow(nuclei_mask, label_text=f"Nuclei mask after merging: {np.unique(nuclei_mask)}")
        assert np.max(nuclei_masks) > 0 and np.max(cell_masks) > 0, f"No nuclei or cell mask found for {image_path}"
        np.save(image_path / "images.npy", images)
        np.save(image_path / "nuclei_masks.npy", nuclei_masks)
        np.save(image_path / "cell_masks.npy", cell_masks)
        images_paths.append(image_path / "images.npy")
        nuclei_mask_paths.append(image_path / "nuclei_masks.npy")
        cell_mask_paths.append(image_path / "cell_masks.npy")
    return image_paths, nuclei_mask_paths, cell_mask_paths

In [ ]:
def clean_segmentations(image_paths): 
    for image_path in image_paths:
            if (image_path / "images.npy").exists():
                os.remove(image_path / "images.npy") 
            if (image_path / "cell_masks.npy").exists():
                os.remove(image_path / "cell_masks.npy")
            if (image_path / "nuclei_masks.npy").exists():
                os.remove(image_path / "nuclei_masks.npy")

clean_segmentations(dev_image_paths)

In [ ]:
# View the dev-dataset directory before generating the masks and after
!tree $DEV_DATA_DIR

In [ ]:
image_paths, nuclei_mask_paths, cell_mask_paths = get_masks(segmentator, dev_image_paths)
print(nuclei_mask_paths, cell_mask_paths)
!tree $DEV_DATA_DIR

In [ ]:
for nuclei_path, cell_path, image_path in zip(nuclei_mask_paths, cell_mask_paths, image_paths):
    sample_nuclei_mask = np.load(nuclei_path)
    sample_nuclei = cv2.imread(str(image_path / (CHANNEL_NAMES[DAPI] + ".png")), cv2.IMREAD_UNCHANGED)
    microshow(sample_nuclei_mask)
    microshow(sample_nuclei)

    sample_cell_mask = np.load(cell_path)
    sample_microtuble = cv2.imread(str(image_path / (CHANNEL_NAMES[TUBL] + ".png")), cv2.IMREAD_UNCHANGED)
    microshow(sample_cell_mask)
    microshow(sample_microtuble)

In [ ]:
def merge_nuclei(nuclei_mask, cell_mask, dialation_radius=20):
    bin_nuc_mask = (nuclei_mask > 0).astype(np.int8)
    cls_nuc = morphology.closing(bin_nuc_mask, morphology.disk(dialation_radius))
    # get the labels of touching nuclei
    new_label_map = morphology.label(cls_nuc)
    new_label_idx = np.unique(new_label_map)[1:]

    new_cell_mask = np.zeros_like(cell_mask)
    new_nuc_mask = np.zeros_like(nuclei_mask)
    for new_label in new_label_idx:
        # get the label of the touching nuclei
        old_labels = np.unique(nuclei_mask[new_label_map == new_label])
        old_labels = old_labels[old_labels != 0]

        new_nuc_mask[np.isin(nuclei_mask, old_labels)] = new_label
        new_cell_mask[np.isin(cell_mask, old_labels)] = new_label

        # for old_label in old_labels:
        #     new_cell_mask[cell_mask == old_label] = new_label
        #     new_nuc_mask[nuclei_mask == old_label] = new_label
    return new_nuc_mask, new_cell_mask


def clean_cell_masks(
    cell_mask,
    nuclei_mask,
    rm_border=True, # removes cells touching the border
    remove_size=1000, # remove cells smaller than remove_size, based on the area of the bounding box, honestly could be higher, mb 2500. Make 0 to turn off.
    dialation_radius=20, # this is for 2048x2048 images adjust as needed. Make 0 to turn off.
    # final_size=None, # resize the mask to final_size
):
    num_removed = 0
    if rm_border:
        cleared_nuclei_mask = segmentation.clear_border(nuclei_mask)
        keep_value = np.unique(cleared_nuclei_mask)
        bordering_cells = np.array([[x_ in keep_value for x_ in x] for x in cell_mask]).astype("uint8")
        cleared_cell_mask = cell_mask * bordering_cells
        num_removed = len(np.unique(nuclei_mask)) - len(keep_value)  # -1 because 0 is not a cell
        if num_removed == np.max(nuclei_mask):
            assert np.max(keep_value) == 0, f"Something went wrong with clearing the border, num_removed is the same as the highest index mask in nuclei mask, but the keep_value {np.max(keep_value)} != 0"
        nuclei_mask = cleared_nuclei_mask
        cell_mask = cleared_cell_mask

    assert set(np.unique(nuclei_mask)) == set(np.unique(cell_mask))

    # needs to be after clear border otherwise you get a boundary of nuclei that are still touching the edge
    # maybe that is due to the interpolation method
    # ideally this happens outside
    # if final_size is not None:
    #     nuclei_mask = cv2.resize(nuclei_mask, final_size, interpolation=cv2.INTER_NEAREST)
    #     cell_mask = cv2.resize(cell_mask, final_size, interpolation=cv2.INTER_NEAREST)

    ### see if nuclei are touching and merge them
    if dialation_radius > 0:
        nuclei_mask, cell_mask = merge_nuclei(nuclei_mask, cell_mask, dialation_radius=dialation_radius)
        assert set(np.unique(nuclei_mask)) == set(np.unique(cell_mask))
    else:
        cell_mask = cell_mask
        nuclei_mask = nuclei_mask

    region_props = measure.regionprops(cell_mask, (cell_mask > 0).astype(np.uint8))
    pre_size = len(region_props)
    if remove_size > 0:
        region_props = [x for x in region_props if x.area > remove_size]
        num_removed += pre_size - len(region_props)
    bbox_array = np.array([x.bbox for x in region_props])

    # convert x1,y1,x2,y2 to x,y,w,h
    bbox_array[:, 2] = bbox_array[:, 2] - bbox_array[:, 0]
    bbox_array[:, 3] = bbox_array[:, 3] - bbox_array[:, 1]

    # com_array = np.array([x.weighted_centroid for x in region_props])

    return cell_mask, nuclei_mask, bbox_array, num_removed

In [ ]:
total_num_removed = 0
total_num_original = 0
for cell_mask_path, nuclei_mask_path, image_path in zip(cell_mask_paths, nuclei_mask_paths, viz_image_paths):
    elements = np.load(cell_mask_path)
    targets = np.load(nuclei_mask_path)
    microplot.microshow(targets)
    microplot.microshow(elements)
    # cell_mask, nuclei_mask, _, n_removed, n_original = save_single_cell_image(cell_mask, nuclei_mask)
    n_original = len(np.unique(targets))
    elements, targets, bbox_array, n_removed = clean_cell_masks(
        elements, targets, rm_border=True, remove_size=2500, dialation_radius=0
    )
    total_num_removed += n_removed
    total_num_original += n_original
    microplot.microshow(targets)
    microplot.microshow(elements)
    cell_image = np.asarray(cv2.imread(str(image_path / (CHANNEL_NAMES[TUBL] + ".png")), cv2.IMREAD_UNCHANGED))
    nuclei_image = np.asarray(cv2.imread(str(image_path / (CHANNEL_NAMES[DAPI] + ".png")), cv2.IMREAD_UNCHANGED))
    image = np.stack([cell_image, nuclei_image])
    microplot.microshow(image, cmaps=["pure_red", "pure_blue"])
    microplot.microshow(image * (elements > 0)[None, ...], cmaps=["pure_red", "pure_blue"])
print("fraction removed:", total_num_removed / total_num_original)

In [ ]:
def save_single_cell_image(
    cell_mask,
    nuclei_mask,
    rm_border=True, # removes cells with nuclei touching the border
    remove_size=2500, # remove cells smaller than remove_size, based on the area of the bounding box, honestly could be higher, mb 2500. Make 0 to turn off.
    dialation_radius=0, # this is for 2048x2048 images adjust as needed. Make 0 to turn off.
):
    num_original = len(np.unique(nuclei_mask))
    new_cell_mask, new_nuc_mask, bbox_array, num_removed = clean_cell_masks(
        cell_mask,
        nuclei_mask,
        rm_border=rm_border,
        remove_size=remove_size,
        dialation_radius=dialation_radius,
    )
    return new_cell_mask, new_nuc_mask, bbox_array, num_removed, num_original

